In [35]:
import json
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
from config import snowflake_conn_prop
import pandas as pd

session = Session.builder.configs(snowflake_conn_prop).create()

In [36]:
# how I would do in pandas
df = pd.read_csv('./medium_data.csv', index_col=0)
df.head(5)
# 6508 urls

,url,title,subtitle,image,claps,responses,reading_time,publication,date
id,,,,,,,,,
1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,NaN,1.png,850,8,8,Towards Data Science,2019-05-30
2,https://towardsdatascience.com/hands-on-graph-...,Hands-on Graph Neural Networks with PyTorch & ...,NaN,2.png,1100,11,9,Towards Data Science,2019-05-30
3,https://towardsdatascience.com/how-to-use-ggpl...,How to Use ggplot2 in Python,A Grammar of Graphics for Python,3.png,767,1,5,Towards Data Science,2019-05-30
4,https://towardsdatascience.com/databricks-how-...,Databricks: How to Save Files in CSV on Your L...,When I work on Python projects dealing…,4.jpeg,354,0,4,Towards Data Science,2019-05-30
5,https://towardsdatascience.com/a-step-by-step-...,A Step-by-Step Implementation of Gradient Desc...,One example of building neural…,5.jpeg,211,3,4,Towards Data Science,2019-05-30


In [37]:
session.file.put('./medium_data.csv', '@~/', overwrite=True)
schema = StructType([
  StructField("id", IntegerType()),
  StructField("url", StringType()),
  StructField("title", StringType()),
  StructField("subtitle", StringType()),
  StructField("image", StringType()),
  StructField("claps", IntegerType()),
  StructField("responses", VariantType()),
  StructField("reading_time", IntegerType()),
  StructField("publication", StringType()),
  StructField("date", DateType())
])
df = session.read.option("skip_header", 1).option("field_delimiter", ",").option("FIELD_OPTIONALLY_ENCLOSED_BY", "\"").schema(schema).csv('@~/medium_data.csv')
df_pd = df.to_pandas().set_index('ID')

In [39]:
df_pd = df_pd.loc[~df_pd["URL"].str.startswith('https://')]

,URL,TITLE,SUBTITLE,IMAGE,CLAPS,RESPONSES,READING_TIME,PUBLICATION,DATE
ID,,,,,,,,,


In [33]:
df_tldr = df.select(
    df.url, df.title, call_builtin('url_resolve_and_summarize', df.url).alias("TL_DR")
    )

df_tldr.write.save_as_table('medium_tldr', mode="overwrite")